In [1]:
#install reuired libraries
!pip install --upgrade tables
!pip install eli5
!pip install 
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 
ERROR: You must give at least one requirement to install (see "pip help install")


In [0]:
#impor libraries
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

#for optymalisation
from hyperopt import hp,fmin,tpe,STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/car_price/car_price/"

/content/drive/My Drive/Colab Notebooks/matrix/car_price/car_price


In [6]:
#read data
df=pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
#factorise values
suf='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list):continue

  factorised_values=df[feat].factorize()[0]

  if suf in feat:
    df[feat]=factorised_values
  else:
    df[feat+suf]=factorised_values

In [9]:
# factorised columns without price
cat_feats= [x for x in df.columns if suf in x]
cat_feats= [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
#change to int
df['param_rok-produkcji__cat']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))
df['param_moc__cat']=df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa__cat']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(x.split('cm')[0].replace(' ', '')))

In [0]:
#function for model
def run_model(model,feats):
  #new X, y
  X=df[feats].values
  y=df['price_value'].values

  #crate, train and test model
  model=model
  scores=cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
xgb_params={
    'max_depth':7,
    'n_estimators':100, 
    'seed':0, 
    'learning_rate':1,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror'

}

model=xgb.XGBRegressor(**xgb_params)

#features based on PermutationImportance
feats=['param_napęd__cat'
,'param_rok-produkcji__cat'
,'param_stan__cat'
,'param_skrzynia-biegów__cat'
,'param_faktura-vat__cat'
,'param_moc__cat'
,'param_marka-pojazdu__cat'
,'feature_kamera-cofania__cat'
,'param_typ__cat'
,'param_pojemność-skokowa__cat'
,'seller_name__cat'
,'feature_wspomaganie-kierownicy__cat'
,'param_model-pojazdu__cat'
,'param_wersja__cat'
,'param_kod-silnika__cat'
,'feature_system-start-stop__cat'
,'feature_asystent-pasa-ruchu__cat'
,'feature_czujniki-parkowania-przednie__cat'
,'feature_łopatki-zmiany-biegów__cat'
,'feature_regulowane-zawieszenie__cat']

In [35]:
#run model with static parameters
run_model(model, feats)

(-9592.565449925363, 47.480997093379706)

In [0]:
# function for getting loss(mean_mae) and status

def obj_func(params):
  print("Training params:")
  print(params)

  mean_mae,score_std=run_model(xgb.XGBRegressor(**xgb_params), feats)

  return{'loss': np.abs(mean_mae), 'status': STATUS_OK}

In [37]:
#space
xgb_reg_params= {
    'learning_rate':    hp.choice('learning rate', np.arange(0.05,0.31,0.05)),
    'max_depth':        hp.choice('max_depth', np.arange(5,16,1, dtype=int)),
    'subsample':        hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree': hp.quniform('colsample_bytree',0.5,1,0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             0
}
#run
best=fmin(obj_func,xgb_reg_params,algo=tpe.suggest,max_evals=25,return_argmin=False)
best

Training params:
{'colsample_bytree': 0.65, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training params:
{'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training params:
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training params:
{'colsample_bytree': 0.75, 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
Training params:
{'colsample_bytree': 0.75, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training params:
{'colsample_bytree': 0.65, 'learning_rate': 0.15000000000000002, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'se

{'colsample_bytree': 0.65,
 'learning_rate': 0.05,
 'max_depth': 8,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'seed': 0,
 'subsample': 0.55}